In [1]:
from numba import cuda
import math
import numpy as np
import time

In [35]:
@cuda.jit
def increment_by_one(input_val, results_arr):
    i = cuda.grid(1)
    if i < input_val.size:
        results_arr[i] = input_val[i] + 1
        
results_arr = cuda.device_array((np.full((40), 1)), np.float32)
data = np.arange(40, dtype=np.float32)
increment_by_one[1, 1](data, results_arr)

#c = results_arr.copy_to_host()


/opt/conda/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/opt/conda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [36]:
c = results_arr.copy_to_host()

ValueError: maximum supported dimension for an ndarray is 32, found 40

In [37]:
@cuda.jit
def add_array(a, b, c):
    i = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    if i < a.size:
        c[i] = a[i] + b[i]

N = 20
a = np.arange(N, dtype=np.float32)
b = np.arange(N, dtype=np.float32)
dev_c = cuda.device_array_like(a)

add_array[4, 8](a, b, dev_c)

c = dev_c.copy_to_host()
print(c)

[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38.]


/opt/conda/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/opt/conda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [8]:
from numba import cuda
import math
import numpy as np
import time

@cuda.jit
def compute_z(data, mean, std, results):
    i = cuda.grid(1)
    if i < data.size:
        if 1 > 50000000:
            print(i)
        results[i] = (data[i] - mean[0]) / std[0]

#LOAD DATA
data = np.random.randint(50, 5000, (100000000,)).astype(np.float32)
results = np.full((data.shape[0]), np.nan).astype(np.float32)
data_mean, data_std = np.array([np.mean(data)]).astype(np.float32), np.array([np.mean(data)]).astype(np.float32)

#SEND DATA TO GPU
start_time = time.time()
device_data = cuda.to_device(data)
device_mean = cuda.to_device(data_mean)
device_std = cuda.to_device(data_std)
device_results = cuda.device_array_like(results)

#RUN ON GPU
threadsperblock = 256
blocks_per_grid = math.ceil(data.shape[0] / threadsperblock)
compute_z[blocks_per_grid, threadsperblock](device_data, device_mean, device_std, device_results)
results = device_results.copy_to_host()

print(time.time() - start_time)

0.3935701847076416


In [7]:
result = device_results.copy_to_host()

In [6]:
threadsperblock

256

In [8]:
result


array([-0.19867931,  0.80782384, -0.75797975, ...,  0.28219232,
       -0.50882965, -0.38801757], dtype=float32)

In [116]:
data

array([9., 4., 0., 8., 3., 4., 4., 1., 1., 4., 8., 4., 3., 5., 6., 6., 7.,
       0., 5., 9., 4., 3., 6., 2., 3., 0., 3., 8., 2., 3., 6., 0., 6., 7.,
       7., 6., 2., 8., 6., 7., 7., 2., 7., 0., 3., 8., 7., 2., 8., 9., 9.,
       8., 7., 1., 9., 0., 6., 2., 1., 0., 0., 8., 7., 4., 8., 4., 6., 8.,
       4., 7., 6., 4., 2., 5., 8., 9., 5., 5., 2., 0., 1., 6., 2., 4., 4.,
       4., 3., 6., 5., 0., 3., 1., 9., 6., 3., 8., 7., 1., 0., 1.],
      dtype=float32)

In [51]:
from numba import vectorize, float64
import numpy as np

@vectorize([float64(float64)])
def compute_z(x):
    return x**2
    
data = np.random.randint(50, 5000, (10,)).astype(np.float64)
results = compute_z(data=data)




TypeError: compute_z() takes from 1 to 2 positional arguments but 0 were given

In [39]:
from numba import vectorize, float64

@vectorize([float64(float64, float64)])
def f(x, y):
    return x + y

In [40]:
a = np.arange(6)
f(a, a)

array([ 0.,  2.,  4.,  6.,  8., 10.])

In [7]:
from numba import vectorize, cuda, float64, njit, prange
import numpy as np
import time

@vectorize(['float64(float64)'], target='cuda')
def compute_z_gpu(x):
    return x**2

@njit('(float64[:],)')
def compute_z_cpu(data):
    results = np.full((data.shape[0]), np.nan)
    for i in prange(results.shape[0]):
        results[i] = data[i]**2

data = np.random.randint(50, 5000, (100000000,)).astype(np.float64)

start = time.time()
results_gpu = compute_z_gpu(data)
print(time.time() - start)


start = time.time()
results_cpu = compute_z_cpu(data=data)
print(time.time() - start)






0.6380696296691895
0.38841986656188965


In [96]:
from numba import cuda
import math
import numpy as np
import time

threadsperblock = 256

@cuda.jit
def sliding_ttest(data, stride, output):
    start = cuda.grid(1)
    stride_len = int(stride[0])
    if start < (data.shape[0] - stride_len):
        sample_1 = data[start:start+stride_len][0]
        sample_2 = data[start:start+stride_len][1]
        sample_1_sum = 0; sample_2_sum = 0; sample_1_deviation = 0; sample_2_deviation = 0
        for i in sample_1: sample_1_sum += i
        for i in sample_2: sample_2_sum += i
        sample_1_mean = sample_1_sum / sample_1.shape[0]
        sample_2_mean = sample_2_sum / sample_2.shape[0]
        for i in sample_1: sample_1_deviation += (sample_1_mean - i)**2
        for i in sample_2: sample_2_deviation += (sample_2_mean - i)**2
        sample_1_stdev = sample_1_deviation / sample_1.shape[0]
        sample_2_stdev = sample_2_deviation / sample_2.shape[0]
        pooled_std = math.sqrt((len(sample_1) - 1) * sample_1_stdev **2 + (len(sample_2) - 1) * sample_2_stdev**2) / (len(sample_1) - 1) + (len(sample_2) - 1)
        
        result = (sample_1_mean - sample_2_mean) / (pooled_std * math.sqrt(1 / len(sample_1) + 1 / len(sample_1)))
        #output[start] = results
        output[start] += result
        
start = time.time()
sample_1 = np.random.randint(0, 10, (50000000,))
sample_2 = np.random.randint(0, 10, (50000000,))
sample_data = np.vstack((sample_1, sample_2)).T

results = np.full((sample_1.shape[0]), np.nan).astype(np.float64)

device_data = cuda.to_device(sample_data)
device_stride = cuda.to_device(np.array([10]).astype(np.float32))
device_results = cuda.device_array_like(results)

blocks_per_grid = math.ceil(sample_data.shape[0] / threadsperblock)
sliding_ttest[blocks_per_grid, threadsperblock](device_data, device_stride, device_results)
results = device_results.copy_to_host()
print(time.time() - start)

#device_data = cuda.to_device(data)


3.034396171569824


array([ 2.95518450e+000, -4.49843834e-001, -9.94491992e-002,
        1.58857866e+000, -2.00000000e+000,  1.53846154e-001,
       -3.06386978e-001,  4.13508039e-001,  1.78138572e-001,
       -7.80776406e-001,  2.07106781e+000, -1.23105626e+000,
        9.19160934e-001, -9.35689300e-002, -4.67844650e-002,
       -4.78242305e-001,  2.97289050e-001, -3.38581381e-001,
       -5.00000000e-001,  5.20000000e+000,  1.97626258e-323,
       -8.05096808e-001,  1.84351204e-001, -9.94491992e-002,
       -1.44416242e-001,  1.22554791e+000,  1.48219694e-323,
       -1.78138572e-001, -4.00000000e-001,  4.61538462e-001,
       -9.19160934e-001,  1.37836013e-001,  4.06552208e-001,
       -5.34415717e-001,  4.50663314e-001,  3.90388203e-001,
       -1.98898398e-001, -2.43261944e-001,  7.43222625e-002,
        1.35572333e-001,  4.13508039e-001, -4.99220728e-001,
       -2.96897620e-001,  6.13957761e-001, -1.01091369e+000,
       -9.19160934e-001,  3.49878538e-001, -1.45957167e-001,
        7.17363457e-001,

In [1]:
from numba import cuda, njit
import maths
import numpy as np
import time

In [7]:



from numba import cuda
import math
import numpy as np
import time

threadsperblock = 256

@cuda.jit
def sliding_ttest(data, stride, output):
    start = cuda.grid(1)
    stride_len = int(stride[0])
    if start < (data.shape[0] - stride_len):
        sample_1 = data[start:start+stride_len][0]
        sample_2 = data[start:start+stride_len][1]
        sample_1_sum = 0; sample_2_sum = 0; sample_1_deviation = 0; sample_2_deviation = 0
        for i in sample_1: sample_1_sum += i
        for i in sample_2: sample_2_sum += i
        sample_1_mean = sample_1_sum / sample_1.shape[0]
        sample_2_mean = sample_2_sum / sample_2.shape[0]
        for i in sample_1: sample_1_deviation += (sample_1_mean - i)**2
        for i in sample_2: sample_2_deviation += (sample_2_mean - i)**2
        sample_1_stdev = sample_1_deviation / sample_1.shape[0]
        sample_2_stdev = sample_2_deviation / sample_2.shape[0]
        pooled_std = math.sqrt((len(sample_1) - 1) * sample_1_stdev **2 + (len(sample_2) - 1) * sample_2_stdev**2) / (len(sample_1) - 1) + (len(sample_2) - 1)
        
        result = (sample_1_mean - sample_2_mean) / (pooled_std * math.sqrt(1 / len(sample_1) + 1 / len(sample_1)))
        #output[start] = results
        output[start] += result
        
start = time.time()
sample_1 = np.random.randint(0, 10, (50000000,))
sample_2 = np.random.randint(0, 10, (50000000,))
sample_data = np.vstack((sample_1, sample_2)).T

results = np.full((sample_1.shape[0]), np.nan).astype(np.float64)

device_data = cuda.to_device(sample_data)
device_stride = cuda.to_device(np.array([10]).astype(np.float32))
device_results = cuda.device_array_like(results)

blocks_per_grid = math.ceil(sample_data.shape[0] / threadsperblock)
sliding_ttest[blocks_per_grid, threadsperblock](device_data, device_stride, device_results)
results = device_results.copy_to_host()
print(time.time() - start)

sample_1 = np.random.randint(1, 100, (10, 2))
sample_2 = np.random.randint(1, 100, (5, 2))
results = device_results = cuda.device_array_like(results)

device_sample_1 = cuda.to_device(sample_1)
device_sample_2 = cuda.to_device(sample_2)

In [14]:
from numba import cuda, njit, prange, types
import math
import numpy as np
from typing import Literal, Optional
import time

THREADSPERBLOCK = 256

@cuda.jit
def _cdist_gpu(sample_1: np.ndarray, 
               sample_2: np.ndarray, 
               results: np.ndarray):
    
    """
    Helper to computes the Euclidean distance between every observation in sample_1 and sample_2 on the GPU.
    Called by ``cdist``.
    
    :param sample_1: Two-dimensional array representing data observations (e.g, all data)
    :param sample_2: Two-dimensional array representing a sub-group of observations (e.g., under 18s)
    :param results: Two-dimensional array of size len(sample_1) x len(sample_2) to store the computed distances.
    """
    
    idx = cuda.grid(1)
    if idx < sample_1.shape[0]:
        source = sample_1[idx]
        for j in range(sample_2.shape[0]):
            destination = sample_2[j]
            results[idx, j] = math.sqrt((destination[0] - source[0]) ** 2 + (destination[1] - source[1]) ** 2)
    
@njit('(float32[:, :], float32[:, :],)')
def _cdist_cpu(sample_1: np.ndarray, 
               sample_2: np.ndarray) -> np.ndarray:
    """
    Helper to computes the Euclidean distance between every observation in sample_1 and sample_2 in parallel using CPU.
    Called by ``cdist``.
    
    :param sample_1: Two-dimensional array representing data observations (e.g, all data)
    :param sample_2: Two-dimensional array representing a sub-group of observations (e.g., under 18s)
    :param results: Two-dimensional array of size len(sample_1) x len(sample_2) to store the computed distances.
    """
    
    results = np.full((sample_1.shape[0], sample_2.shape[0]), np.nan)
    for i in prange(sample_1.shape[0]):
        source = sample_1[i]
        for j in range(sample_2.shape[0]):
            destination = sample_2[j]
            results[i, j] = math.sqrt((destination[0] - source[0]) ** 2 + (destination[1] - source[1]) ** 2)
    return results
            

@njit('(float32[:, :], int64, types.unicode_type)')
def _agg_dists(data: np.ndarray, k: int, agg_method: Literal['mean', 'median']) -> np.ndarray:
    """
    Helper to aggregates distances for each observation based on the k most proximal neighbors.
    Called by ``cdist``.
    
    :param data: Two-dimensional array representing computed distances.
    :param k: The number of most proximal neighbors to consider.
    :param agg_method: The method used to compute the aggregate statistic ('mean' or 'median').
    :return: One-dimensional array containing the aggregated distances.
    """
    
    results = np.full((data.shape[0]), np.nan)
    for i in prange(data.shape[0]):
        sliced_arr = data[i][np.argsort(data[i])][:k]
        if agg_method == 'mean':
            results[i] = np.mean(sliced_arr)
        else:
            results[i] = np.median(sliced_arr)
    return results

    

def cdist(sample_1: np.ndarray, 
          sample_2: np.ndarray, 
          k: int, 
          agg_method: Literal['mean', 'median'] = 'median', 
          backend: Literal['cpu', 'gpu'] = 'cpu') -> np.ndarray:
    
    """
    Computes the aggregate distance to most proximal N observations in sample_2 for every observation in sample 1.
    
    Computetes the Euclidean distance between every observation in sample_1 and sample_2. Next, finds aggretate 
    (mean or median) distance to the N most proximal observations in sample_2 for every observation in sample_1. 
    
    :parameter ndarray sample_1: Two dimensional array representing all data where every row is an observation and columns represent dimensionality reduction features, e.g., UMAP ``X`` and ``Y``.
    :parameter ndarray sample_2: Two dimensional array representing a sub-group (e.g., under-18s) where every row is an observation and columns represent dimensionality reduction features, e.g., UMAP ``X`` and ``Y``.
    :parameter int k: The number of most proximal neighbours to consider when computing the aggregate statistics. 
    :parameter Literal['mean', 'median'] agg_method: The method compute the aggregate statistic. Default: ``median``.
    :parameter Literal['cpu', 'gpu'] backend: If the search should be run on the cpu or gpu. Default: ``cpu``. 
    
    :example 1:
    >>> entire_dataset = np.array([[0, 1], [0, 2], [0, 3]])
    >>> under_18 = np.array([[0, 3], [0, 10]])
    >>> cdist(sample_1=sample_1, sample_2=sample_2, k=2, backend='cpu') 
    >>> [5.5, 4.5, 3.5]
    """
    
    if backend == 'gpu':
        device_sample_1 = cuda.to_device(sample_1.astype(np.float32))
        device_sample_2 = cuda.to_device(sample_2.astype(np.float32))
        start_time = time.time()
        #results = np.full((sample_1.shape[0], sample_2.shape[0]), np.nan).astype(np.float32)
        results = np.empty((sample_1.shape[0], sample_2.shape[0]))
        print(f'NUMBA np.full: {time.time() - start_time}s')
        start_time = time.time()
        device_results = cuda.device_array_like(results)
        print(f'NUMBA device_array_like: {time.time() - start_time}s')
        blocks_per_grid_x = math.ceil((sample_1.shape[0] + sample_2.shape[0]) / THREADSPERBLOCK)
        blocks_per_grid_y = math.ceil(sample_2.shape[0] / THREADSPERBLOCK)
        blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)
        start_time = time.time()
        _cdist_gpu[blocks_per_grid_x, THREADSPERBLOCK](device_sample_1, device_sample_2, device_results)
        print(f'NUMBA _cdist_gpu: {time.time() - start_time}s')
        start_time = time.time()
        results = device_results.copy_to_host()
        print(f'NUMBA copy_to_host: {time.time() - start_time}s')
    
    else:
        results = _cdist_cpu(sample_1=sample_1.astype(np.float32), sample_2=sample_2.astype(np.float32))
    
    #return 0
    return _agg_dists(data=results.astype(np.float32), k=k, agg_method=agg_method)

sample_1 = np.random.randint(0, 50, (5000, 2)).astype(np.float32)
sample_2 = np.random.randint(0, 50, (7500, 2)).astype(np.float32)

start_time = time.time()
gpu_results = cdist(sample_1=sample_1, sample_2=sample_2, k=10, backend='gpu')
gpu_time = time.time() - start_time
start_time = time.time()
cpu_results = cdist(sample_1=sample_1, sample_2=sample_2, k=10, backend='cpu') 
cpu_time = time.time() - start_time
print(f'NUMBA CPU TIME: {cpu_time}s')
print(f'NUMBA GPU TIME: {gpu_time}s')

NUMBA np.full: 4.1961669921875e-05s
NUMBA device_array_like: 0.0006918907165527344s
NUMBA _cdist_gpu: 0.1877901554107666s


/opt/conda/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 49 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


NUMBA copy_to_host: 0.14725923538208008s
NUMBA CPU TIME: 3.7266530990600586s
NUMBA GPU TIME: 3.9617702960968018s


In [15]:
len(gpu_results)


5000

In [16]:
len(cpu_results)

5000